In [69]:
 !pip install folium

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [84]:
import pandas as pd
from datetime import datetime
from datetime import timedelta
import json
import requests
import time
import folium
from folium.plugins import HeatMap

Retrieving and Filtering Dataset

In [24]:
url = 'https://data.nsw.gov.au/data/dataset/aefcde60-3b0c-4bc0-9af1-6fe652944ec2/resource/21304414-1ff1-4243-a5d2-f52778048b29/download/confirmed_cases_table1_location.csv'  

df = pd.read_csv(url)

df = df[['notification_date', 'postcode', 'lga_name19']]
df

,notification_date,postcode,lga_name19
0,2020-01-25,2134,Burwood (A)
1,2020-01-25,2121,Parramatta (C)
2,2020-01-25,2071,Ku-ring-gai (A)
3,2020-01-27,2033,Randwick (C)
4,2020-03-01,2077,Hornsby (A)
...,...,...,...
973407,2022-02-07,2283,Lake Macquarie (C)
973408,2022-02-07,2019,Bayside (A)
973409,2022-02-07,2076,Ku-ring-gai (A)
973410,2022-02-07,2760,Penrith (C)


In [25]:
for i, row in df.iterrows():
  df.at[i, 'notification_date'] = datetime.strptime(df.at[i, 'notification_date'], '%Y-%m-%d')

df

,notification_date,postcode,lga_name19
0,2020-01-25 00:00:00,2134,Burwood (A)
1,2020-01-25 00:00:00,2121,Parramatta (C)
2,2020-01-25 00:00:00,2071,Ku-ring-gai (A)
3,2020-01-27 00:00:00,2033,Randwick (C)
4,2020-03-01 00:00:00,2077,Hornsby (A)
...,...,...,...
973407,2022-02-07 00:00:00,2283,Lake Macquarie (C)
973408,2022-02-07 00:00:00,2019,Bayside (A)
973409,2022-02-07 00:00:00,2076,Ku-ring-gai (A)
973410,2022-02-07 00:00:00,2760,Penrith (C)


In [56]:
current_date = datetime.now()
N_days_ago = 365 #CHANGE THIS VARIABLE TO CHANGE THE TIMEFRAME

filter_date = current_date - timedelta(days=N_days_ago)

working_data = df[df.notification_date > filter_date]

working_data

,notification_date,postcode,lga_name19
693455,2022-01-19 00:00:00,2443,Port Macquarie-Hastings (A)
693456,2022-01-19 00:00:00,2761,Blacktown (C)
693457,2022-01-19 00:00:00,2176,Fairfield (C)
693458,2022-01-19 00:00:00,2259,Central Coast (C) (NSW)
693459,2022-01-19 00:00:00,2286,Lake Macquarie (C)
...,...,...,...
973407,2022-02-07 00:00:00,2283,Lake Macquarie (C)
973408,2022-02-07 00:00:00,2019,Bayside (A)
973409,2022-02-07 00:00:00,2076,Ku-ring-gai (A)
973410,2022-02-07 00:00:00,2760,Penrith (C)


In [30]:
#working_data = df.groupby(["lga_name19"]).count().transform("count")
#working_data
#working_data['count'] = working_data.groupby("lga_name19")["lga_name19"].transform("count")
#working_data

<ipython-input-30-9bfb66571a75>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  working_data['count'] = working_data.groupby("lga_name19")["lga_name19"].transform("count")


,notification_date,postcode,lga_name19,count
693455,2022-01-19 00:00:00,2443,Port Macquarie-Hastings (A),2704.0
693456,2022-01-19 00:00:00,2761,Blacktown (C),15564.0
693457,2022-01-19 00:00:00,2176,Fairfield (C),6585.0
693458,2022-01-19 00:00:00,2259,Central Coast (C) (NSW),12881.0
693459,2022-01-19 00:00:00,2286,Lake Macquarie (C),7178.0
...,...,...,...,...
973407,2022-02-07 00:00:00,2283,Lake Macquarie (C),7178.0
973408,2022-02-07 00:00:00,2019,Bayside (A),5612.0
973409,2022-02-07 00:00:00,2076,Ku-ring-gai (A),3227.0
973410,2022-02-07 00:00:00,2760,Penrith (C),9350.0


In [57]:
test = working_data
test = test.groupby("lga_name19")["lga_name19"].count()
test.index.name = 'lga'
test = test.reset_index()
test = test.rename(columns={"lga_name19" : "count"})
test

,lga,count
0,Albury (C),1761
1,Armidale Regional (A),1276
2,Ballina (A),1648
3,Balranald (A),74
4,Bathurst Regional (A),1516
...,...,...
125,Wingecarribee (A),1197
126,Wollondilly (A),1590
127,Wollongong (C),7954
128,Woollahra (A),1594


Determining Longitude and Latitude




In [58]:
working_data = test
working_data.reset_index(drop=True, inplace=True)
working_data

working_data['latitude'] = None
working_data['longitude'] = None

working_data

,lga,count,latitude,longitude
0,Albury (C),1761,None,None
1,Armidale Regional (A),1276,None,None
2,Ballina (A),1648,None,None
3,Balranald (A),74,None,None
4,Bathurst Regional (A),1516,None,None
...,...,...,...,...
125,Wingecarribee (A),1197,None,None
126,Wollondilly (A),1590,None,None
127,Wollongong (C),7954,None,None
128,Woollahra (A),1594,None,None


In [64]:
base_url = 'https://nominatim.openstreetmap.org/search'

for i, row in working_data.iterrows():
  location = str(working_data.at[i, 'lga'].rsplit(" ", 1)[0]) 
  my_params= {'q': location + ', NSW, Australia','format':'json','limit':'1'}
  response = requests.get(base_url, params = my_params)
  match = response.json()

  
  longitude = float(match[0]['lon'])
  latitude = float(match[0]['lat'])

  working_data.at[i, 'longitude'] = longitude
  working_data.at[i, 'latitude'] = latitude

  time.sleep(1)



working_data

,lga,count,latitude,longitude
22,Canada Bay (A),2459,-33.848513,151.097582
26,Central Darling (A),62,-31.781445,143.593218
34,Correctional settings,723,-32.819678,151.336354
35,Cowra (A),301,-33.834939,148.692516
36,Cumberland (A),10048,-33.836767,150.967143
...,...,...,...,...
125,Wingecarribee (A),1197,-34.49079,150.366924
126,Wollondilly (A),1590,-34.069557,150.402938
127,Wollongong (C),7954,-34.427808,150.893054
128,Woollahra (A),1594,-33.861834,151.273332


Filtering data based on desired radius

In [65]:
desired_suburb = 'Manly' #CHANGE THIS TO CHANGE THE SUBURB

base_url = 'https://nominatim.openstreetmap.org/search'

my_params= {'q': desired_suburb + ', NSW, Australia','format':'json','limit':'1'}
response = requests.get(base_url, params = my_params)
match = response.json()

desired_longitude = float(match[0]['lon'])
desired_latitude = float(match[0]['lat'])


In [66]:
from math import sin, cos, sqrt, atan2, radians

def calculate_distance(desired_latitude, desired_longitude, latitude, longitude):
# approximate radius of earth in km
  R = 6373.0

  lat1 = radians(latitude)
  lon1 = radians(longitude)
  lat2 = radians(desired_latitude)
  lon2 = radians(desired_longitude)

  dlon = lon2 - lon1
  dlat = lat2 - lat1

  a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
  c = 2 * atan2(sqrt(a), sqrt(1 - a))

  distance = R * c

  return distance

In [67]:
for i, row in working_data.iterrows():
  distance = calculate_distance(desired_latitude, desired_longitude, working_data.at[i, 'latitude'], working_data.at[i, 'longitude'])

  if distance > 50: #CHANGE THIS TO CHANGE THE RADIUS
    working_data.drop(i, inplace=True)

working_data

,lga,count,latitude,longitude
22,Canada Bay (A),2459,-33.848513,151.097582
36,Cumberland (A),10048,-33.836767,150.967143
41,Fairfield (C),6585,-33.86744,150.889322
44,Georges River (A),4893,-34.039242,150.894607
55,Hornsby (A),3304,-33.572408,151.131775
57,Hunters Hill (A),777,-33.829863,151.136011
58,Inner West (A),4705,-33.890017,151.145396
63,Ku-ring-gai (A),3227,-33.727133,151.14921
67,Lane Cove (A),1832,-33.823203,151.169716
71,Liverpool (C),9227,-33.919825,150.92566


In [68]:
working_data.reset_index(drop=True, inplace=True)
working_data

,lga,count,latitude,longitude
0,Canada Bay (A),2459,-33.848513,151.097582
1,Cumberland (A),10048,-33.836767,150.967143
2,Fairfield (C),6585,-33.86744,150.889322
3,Georges River (A),4893,-34.039242,150.894607
4,Hornsby (A),3304,-33.572408,151.131775
5,Hunters Hill (A),777,-33.829863,151.136011
6,Inner West (A),4705,-33.890017,151.145396
7,Ku-ring-gai (A),3227,-33.727133,151.14921
8,Lane Cove (A),1832,-33.823203,151.169716
9,Liverpool (C),9227,-33.919825,150.92566


Visualising Data

In [80]:
m = folium.Map(location = [-33.8, 151.2], tiles ='OpenStreetMap',  zoom_start=11)


for i, row in working_data.iterrows():
    lat = working_data.at[i, 'latitude']
    lng = working_data.at[i, 'longitude']

    folium.Circle(location = [lat, lng], popup = working_data.at[i, "lga"],  color='crimson', fill = True, fill_color = "crimson", radius=float(working_data.at[i, "count"])).add_to(m)

m

In [86]:
hm = folium.Map(location = [-33.8, 151.2], tiles ='OpenStreetMap',  zoom_start=11)
map_values = working_data[["latitude", "longitude", "count"]].values.tolist()

HeatMap(map_values,
                min_opacity=0.05, 
                max_opacity=0.9, 
                radius=25,
                use_local_extrema=False).add_to(hm)

hm